## IDs:
ID 1: 204502926 <br>
ID 2: 039065313

## Imports:

In [1]:
import numpy as np
import pandas as pd
# %matplotlib inline
import matplotlib.pyplot as plt
from collections import defaultdict
import seaborn as sns
from numpy.linalg import solve

np.random.seed(0)

### Question 1: Item Similarity (40 points)

#### a. Build a movie profile vector based on the item features of your choice. Suggested options: item genres, item title

#### b. Build a function which provides the 5 most similar items to a specific item. Item to item similarity is calculated using the cosine similarity metric.

#### c. Use the above function to find most similar items for 2 items from the dataset. Explain your results.

#### d. Use the matrix factorization item representation you built in exercise 1 to find the most similar items for the same 2 items you used above. Compare between the results.

***
### Question 2: Content based recommendation (60 points)

#### a. Build a function which recommends 5 most relevant items to a user. 

### b. Test your recommender system on 2 users. Explain your results.

### c. Test your recommender system on 2 users. Explain your results.